In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

import os

# Agents Example

In [11]:
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    return_intermediate_steps=True,
)

In [14]:
from langchain.callbacks import StdOutCallbackHandler

response = agent(
    {"input": "What is the current American president's age? Return the age multiplied by 82"},
    callbacks=[StdOutCallbackHandler()],
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Wikipedia, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is the current American president's age? Return the age multiplied by 82
Though

In [15]:
response["intermediate_steps"][0][0]

AgentAction(tool='Wikipedia', tool_input='Current American president age', log='I can use Wikipedia to find the current American president\'s age and then calculate the result by multiplying it by 82.\nAction: Wikipedia\nAction Input: "Current American president age"')

In [16]:
import wikipedia

wikipedia.search("Current American president")

['President of the United States',
 'List of presidents of the United States',
 'List of current presidents of legislatures',
 'List of current vice presidents and designated acting presidents',
 'President of Harvard University',
 'Abraham Lincoln',
 'George Washington',
 'Vice President of the United States',
 'Joe Biden',
 'Thomas Jefferson']

In [17]:
wikipedia.summary("List of US presidents by age")

'In this list of presidents of the United States by age, the first table charts the age of each president of the United States at the time of presidential inauguration (first inauguration if elected to multiple and consecutive terms), upon leaving office, and at the time of death. Where the president is still living, their lifespan and post-presidency timespan are calculated up to January 12, 2024.'

In [20]:
from langchain.chains import LLMMathChain

chain = LLMMathChain.from_llm(llm, verbose=True)

In [21]:
chain.run("45 * 69", callbacks=[StdOutCallbackHandler()])



> Entering new LLMMathChain chain...
45 * 69

> Entering new LLMChain chain...
Prompt after formatting:
Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${Question with math problem.}
```text
${single line mathematical expression that solves the problem}
```
...numexpr.evaluate(text)...
```output
${Output of running the code}
```
Answer: ${Answer}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: 45 * 69


> Finished chain.
```text
45 * 69
```
...numexpr.evaluate("45 * 69")...

Answer: 3105
> Finished chain.


'Answer: 3105'

# Building a custom tool

In [22]:
from langchain.document_loaders import PyPDFLoader

file_path = "./mixed_data/ESLII_print12_toc (1).pdf"

loader = PyPDFLoader(file_path)
data = loader.load_and_split()

In [23]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()
docstore = Chroma.from_documents(data, embedding, collection_name="statistical_learning")

In [26]:
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docstore.as_retriever(),
)

In [27]:
chain.run("What is machine learning?", callbacks=[StdOutCallbackHandler()])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
This is page 1
Printer: Opaque this
1
Introduction
Statistical learning plays a key role in many areas of science, ﬁnance and
industry. Here are some examples of learning problems:
•Predict whether a patient, hospitalized due to a heart attac k, will
have a second heart attack. The prediction is to be based on de mo-
graphic, diet and clinical measurements for that patient.
•Predict the price of a stock in 6 months from now, on the basis o f
company performance measures and economic data.
•Identify the numbers in a handwritten ZIP code, from a digiti zed
image.
•Estimate the amount of glucose in the blood of a diabetic pers on,
from the infrared absorption spectr

'Machine learning is a field of study that focuses on developing algorithms and models that allow computers to learn and make predictions or decisions based on data. It involves the development of mathematical and statistical models that can automatically analyze and interpret large amounts of data, identify patterns, and make predictions or decisions without being explicitly programmed. Machine learning algorithms are trained on historical data to learn from it and then apply that knowledge to make predictions or decisions on new, unseen data. It is widely used in various fields such as finance, healthcare, marketing, and robotics.'

In [28]:
from langchain.agents import Tool

description = """
useful for when you need to answer questions
about Machine Learning and Statistical Learning.
Input should be fully formed question.
"""

retrieval_tool = Tool(
    name="Statistical Learning Book",
    description=description,
    func=chain.run,
)

In [31]:
tools = load_tools(['wikipedia', 'llm-math'], llm=llm)

tools.append(retrieval_tool)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    return_intermediate_steps=True,
    verbose=True,
)

In [33]:
response = agent(
    {"input": "Can we use machine learning to predict the age of the president?"},
    callbacks=[StdOutCallbackHandler()],
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.
Statistical Learning Book: 
useful for when you need to answer questions
about Machine Learning and Statistical Learning.
Input should be fully formed question.


Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Wikipedia, Calculator, Statistical Learning Book]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I no